In [1]:
import time
import pandas as pd
import numpy as np
import random
from random import choice
import datetime
# from pivottablejs import pivot_ui
import seaborn as sns
import matplotlib.pyplot as plt

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
new_df = pd.read_csv('bank-full.csv',sep=";")
new_df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


In [3]:
new_df.describe().T

,count,mean,std,min,25%,50%,75%,max
age,45211.0,40.936210,10.618762,18.0,33.0,39.0,48.0,95.0
balance,45211.0,1362.272058,3044.765829,-8019.0,72.0,448.0,1428.0,102127.0
day,45211.0,15.806419,8.322476,1.0,8.0,16.0,21.0,31.0
duration,45211.0,258.163080,257.527812,0.0,103.0,180.0,319.0,4918.0
campaign,45211.0,2.763841,3.098021,1.0,1.0,2.0,3.0,63.0
pdays,45211.0,40.197828,100.128746,-1.0,-1.0,-1.0,-1.0,871.0
previous,45211.0,0.580323,2.303441,0.0,0.0,0.0,0.0,275.0


In [4]:
new_df.isna().sum()

age          0
job          0
marital      0
education    0
default      0
balance      0
housing      0
loan         0
contact      0
day          0
month        0
duration     0
campaign     0
pdays        0
previous     0
poutcome     0
y            0
dtype: int64

In [0]:
### Check if data is balanced or not 

In [6]:
new_df.y.value_counts()

no     39922
yes     5289
Name: y, dtype: int64

In [0]:
### DATA PRE PROCESSING
#### Converting data to more computer readable


In [8]:
new_df.default = new_df.default.apply(lambda x: 1 if x=="yes" else 0 )
new_df.housing = new_df.housing.apply(lambda x: 1 if x=="yes" else 0 )
new_df.loan = new_df.loan.apply(lambda x: 1 if x=="yes" else 0 )
new_df.y = new_df.y.apply(lambda x: 1 if x=="yes" else 0 )
new_df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,0,2143,1,0,unknown,5,may,261,1,-1,0,unknown,0
1,44,technician,single,secondary,0,29,1,0,unknown,5,may,151,1,-1,0,unknown,0
2,33,entrepreneur,married,secondary,0,2,1,1,unknown,5,may,76,1,-1,0,unknown,0
3,47,blue-collar,married,unknown,0,1506,1,0,unknown,5,may,92,1,-1,0,unknown,0
4,33,unknown,single,unknown,0,1,0,0,unknown,5,may,198,1,-1,0,unknown,0


In [0]:
#### Creating a balanced dataframe

In [10]:
indices = []
zeroes = 0
ones_count = int(np.sum(new_df.y))
for i in range(len(new_df)):
    if new_df.y[i] == 0:
        zeroes +=1
        if zeroes > ones_count:
            indices.append(i)

    
balanced_dataset = new_df.drop(indices,axis=0)
len(balanced_dataset)

10578

In [11]:
balanced_dataset.nunique(dropna=False)

age            74
job            12
marital         3
education       4
default         2
balance      3562
housing         2
loan            2
contact         3
day            31
month          12
duration     1436
campaign       33
pdays         423
previous       27
poutcome        4
y               2
dtype: int64

In [0]:
#### GENERATING DUMMIES FOR CATEGORICAL VALUES 

In [13]:
martial = pd.get_dummies(balanced_dataset.marital,drop_first=True,prefix="marital_status",)
education = pd.get_dummies(balanced_dataset.education,drop_first=True,prefix="education")
contact = pd.get_dummies(balanced_dataset.contact,drop_first=True,prefix="contact")
poutcome = pd.get_dummies(balanced_dataset.poutcome,drop_first=True,prefix='poutcome')
df = pd.concat([balanced_dataset,martial,education,contact,poutcome],axis=1,)
df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y,marital_status_married,marital_status_single,education_secondary,education_tertiary,education_unknown,contact_telephone,contact_unknown,poutcome_other,poutcome_success,poutcome_unknown
0,58,management,married,tertiary,0,2143,1,0,unknown,5,may,261,1,-1,0,unknown,0,1,0,0,1,0,0,1,0,0,1
1,44,technician,single,secondary,0,29,1,0,unknown,5,may,151,1,-1,0,unknown,0,0,1,1,0,0,0,1,0,0,1
2,33,entrepreneur,married,secondary,0,2,1,1,unknown,5,may,76,1,-1,0,unknown,0,1,0,1,0,0,0,1,0,0,1
3,47,blue-collar,married,unknown,0,1506,1,0,unknown,5,may,92,1,-1,0,unknown,0,1,0,0,0,1,0,1,0,0,1
4,33,unknown,single,unknown,0,1,0,0,unknown,5,may,198,1,-1,0,unknown,0,0,1,0,0,1,0,1,0,0,1


In [14]:
df.drop(['marital','education','contact','poutcome'],axis=1,inplace=True)
df.head()

,age,job,default,balance,housing,loan,day,month,duration,campaign,pdays,previous,y,marital_status_married,marital_status_single,education_secondary,education_tertiary,education_unknown,contact_telephone,contact_unknown,poutcome_other,poutcome_success,poutcome_unknown
0,58,management,0,2143,1,0,5,may,261,1,-1,0,0,1,0,0,1,0,0,1,0,0,1
1,44,technician,0,29,1,0,5,may,151,1,-1,0,0,0,1,1,0,0,0,1,0,0,1
2,33,entrepreneur,0,2,1,1,5,may,76,1,-1,0,0,1,0,1,0,0,0,1,0,0,1
3,47,blue-collar,0,1506,1,0,5,may,92,1,-1,0,0,1,0,0,0,1,0,1,0,0,1
4,33,unknown,0,1,0,0,5,may,198,1,-1,0,0,0,1,0,0,1,0,1,0,0,1


In [0]:
df.drop(['job','day','month','duration'],axis=1,inplace=True)

In [0]:
X = df.drop(['y'],axis=1)
y  = df['y']

In [0]:
###  SCALING THE DATA 

In [0]:
from sklearn.preprocessing import scale

In [19]:
scaled_x  = scale(X)
scaled_x.shape

(10578, 18)

In [0]:
### SHUFFLING THE DATA 

In [0]:
from sklearn.utils import shuffle

In [0]:
shuffled_scaled_x, shuffled_y = shuffle(scaled_x,y,random_state=101)

In [0]:
### TRAIN, TEST AND VALIDATION WITH BALANCING

In [0]:
total_df = scaled_x.shape[0]
train_data = int(0.8*total_df)
validation_data = int(0.1*total_df)
test_data = int(0.1*total_df)

In [0]:
train_inputs = shuffled_scaled_x[:train_data]
train_outputs = shuffled_y[:train_data]


validation_inputs = shuffled_scaled_x[train_data:train_data+validation_data]
validation_outputs = shuffled_y[train_data:train_data+validation_data]


test_inputs = shuffled_scaled_x[train_data+validation_data:]
test_outputs = shuffled_y[train_data+validation_data:]

In [26]:
print("TRAIN_DATA: ",np.sum(train_outputs)," : ", train_data," : ", np.sum(train_outputs)/train_data)
print("VALIDATION_DATA: ",np.sum(validation_outputs)," : ", validation_data," : ", np.sum(validation_outputs)/validation_data)
print("TEST_DATA: ",np.sum(test_outputs)," : ", test_data," : ", np.sum(test_outputs)/test_data)

TRAIN_DATA:  4258  :  8462  :  0.5031907350508155
VALIDATION_DATA:  516  :  1057  :  0.4881740775780511
TEST_DATA:  515  :  1057  :  0.48722800378429515


In [0]:
np.savez('Bank_data_train',inputs=train_inputs,outputs=train_outputs)
np.savez('Bank_data_validation',inputs=validation_inputs,outputs=validation_outputs)
np.savez('Bank_data_test',inputs=test_inputs,outputs=test_outputs)

In [0]:
### BATCH CREATION

In [0]:
#  Create a class that will do the batching for the alogorithm.


class Batch_creation():
    '''
    This class is a Iterator.
    Dataset is a mandatory argument but batches is optional;
    If the value for batches is given it would automatically take
    the value as None.
    '''

    def __init__(self, dataset, batch_size=None):
        '''
        The dataset that loads would be from train, test or validation.
        So if the input is x('train',5); it will take train.npz with batch size 5
        '''
        npz = np.load(f'Bank_data_{dataset}.npz')

        '''
        Two variables that take the values of the inputs and targets;
        inputs would be floats and targets would be int
        '''
        self.inputs, self.targets = npz['inputs'].astype(
            np.float), npz['outputs'].astype(np.int)

        '''
        Counts the batch number , given by the size you feed in later
        If bacth  size is None , then we are either using validation or testing data , so we take the 
        data in a single batch.
        '''
        if batch_size is None:
            self.batch_size = self.inputs.shape[0]

        else:
            self.batch_size = batch_size
        self.curr_batch = 0
        self.batch_count = self.inputs.shape[0]//self.batch_size

    # method to load the next batch
    def __next__(self):
        if self.curr_batch >= self.batch_count:
            self.curr_batch = 0
            raise StopIteration()

        '''
        slice the dataset in batches and the next method loads them one by one .
        '''
        batch_slice = slice(self.curr_batch * self.batch_size,
                            (self.curr_batch + 1) * self.batch_size)
        inputs_batch = self.inputs[batch_slice]
        targets_batch = self.targets[batch_slice]
        self.curr_batch += 1

        '''
        One hot encode the targest , in this example its not that needed but for futuree refernces.
        '''
        classes_num = 2
        targets_one_hot = np.zeros((targets_batch.shape[0], classes_num))
        targets_one_hot[range(targets_batch.shape[0]), targets_batch] = 1

        '''
        The function would return the inputs batch and the one hot encoded targets
        '''
        return inputs_batch, targets_one_hot

    '''
    An iter class for iterating over the batches, as they would be in a loop.
    An iterator in python is a class with a method __next__ that defines exactly how to iterate through the objects 
    '''

    def __iter__(self):
        return self


In [0]:
# MACHINE LEARNING MODEL

In [0]:
import tensorflow.compat.v1 as tf

In [0]:
input_shape = 18
hidden_shape = 50
output_shape = 2

In [0]:
tf.reset_default_graph()

In [0]:
tf.compat.v1.disable_eager_execution()

In [0]:
#### INPUTS AND TARGETS

In [0]:
inputs = tf.placeholder(tf.float32,[None,input_shape])
targets = tf.placeholder(tf.float32,[None,output_shape])

In [0]:
# ?# HIDDEN LAYER

In [38]:
weights_1 = tf.get_variable('weights_1',[input_shape,hidden_shape])
bias_1 = tf.get_variable('bias_1',[hidden_shape])

output_1 = tf.nn.relu(tf.matmul(inputs,weights_1)+bias_1)

weights_2 = tf.get_variable('weights_2',[hidden_shape,hidden_shape])
bias_2 = tf.get_variable('bias_2',[hidden_shape])

output_2 = tf.nn.relu(tf.matmul(output_1,weights_2)+bias_2)

weights_3 = tf.get_variable('weights_3',[hidden_shape,output_shape])
bias_3 = tf.get_variable('bias_3',[output_shape])

output = tf.matmul(output_2,weights_3)+bias_3

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [39]:
# LOSS

loss = tf.nn.softmax_cross_entropy_with_logits(labels=targets,logits=output)
mean_loss = tf.reduce_mean(loss)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [0]:
# OPTIMISING

optimize = tf.train.AdamOptimizer(learning_rate=0.001).minimize(mean_loss)

In [0]:
# Accuracy 

equals = tf.equal(tf.argmax(targets,1),tf.argmax(output,1))
accuracy = tf.reduce_mean(tf.cast(equals,tf.float32))

In [0]:
sess = tf.compat.v1.Session()


In [0]:
initiliazer = tf.global_variables_initializer()

In [0]:
sess.run(initiliazer)

In [0]:
batch_sizes = 100
max_epochs = 50
pre_validation_loss = 9999999.0

In [0]:
training_data = Batch_creation("train",batch_size=batch_sizes)
validating_data = Batch_creation("validation",batch_size=None)
testing_data = Batch_creation("test",batch_size=None)

In [47]:
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


In [48]:
for epoch_num in range(max_epochs):

  curr_epoch_loss = 0.0

  for input_batch , target_batch in training_data:

    _,batch_loss = sess.run([optimize,mean_loss],
                            feed_dict={inputs:input_batch,targets:target_batch})
    
    curr_epoch_loss += batch_loss

  curr_epoch_loss /= training_data.batch_count

  validation_loss = 0.0
  validation_accuracy = 0.0

  for input_batch, target_batch in validating_data:

    validation_loss,validation_accuracy = sess.run([mean_loss,accuracy],
                                                  feed_dict={inputs:input_batch,targets:target_batch})
    
  print("EPOCH: "+str(epoch_num+1)+" "+
        "TRANING_LOSS:"+' {0:.3f}'.format(curr_epoch_loss)+" "+
        "VALIDATION_LOSS:"+' {0:.3f}'.format(validation_loss)+" "+
        "VALIDATION_ACCURACY:"+' {0:.3f}'.format(validation_accuracy*100.0)+"%")
  
  if validation_loss>pre_validation_loss:
    break
  
  pre_validation_loss = validation_loss

print("END OF TRAINING")

EPOCH: 1 TRANING_LOSS: 0.365791 VALIDATION_LOSS: 0.210842 VALIDATION_ACCURACY: 93.661308%
EPOCH: 2 TRANING_LOSS: 0.167829 VALIDATION_LOSS: 0.192088 VALIDATION_ACCURACY: 93.661308%
EPOCH: 3 TRANING_LOSS: 0.157908 VALIDATION_LOSS: 0.189593 VALIDATION_ACCURACY: 93.661308%
EPOCH: 4 TRANING_LOSS: 0.155019 VALIDATION_LOSS: 0.188918 VALIDATION_ACCURACY: 93.661308%
EPOCH: 5 TRANING_LOSS: 0.153443 VALIDATION_LOSS: 0.188636 VALIDATION_ACCURACY: 93.661308%
EPOCH: 6 TRANING_LOSS: 0.152324 VALIDATION_LOSS: 0.188440 VALIDATION_ACCURACY: 93.661308%
EPOCH: 7 TRANING_LOSS: 0.151472 VALIDATION_LOSS: 0.188280 VALIDATION_ACCURACY: 93.661308%
EPOCH: 8 TRANING_LOSS: 0.150736 VALIDATION_LOSS: 0.188288 VALIDATION_ACCURACY: 93.661308%
END OF TRAINING


In [52]:
for input_batch, target_batch in testing_data:

  test_accuracy = sess.run([accuracy],
                          feed_dict = {inputs:input_batch,targets:target_batch})
testing_acc = test_accuracy[0]*100
print("TRAINING LOSS: "+ "{}".format(testing_acc)+"%")

TRAINING LOSS: 95.4674243927002%
